# Federated PyTorch MNIST Tutorial

In [ ]:
#Install dependencies if not already installed
!pip install torch torchvision

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')
# fx.init('cifar', log_level='METRIC', log_file='./spam_metric.log')

Creating Workspace Directories
Creating Workspace Templates


You should consider upgrading via the '/Users/zhiruzhu/Desktop/data_station/fl_test/venv/bin/python -m pip install --upgrade pip' command.



New workspace directory structure:
workspace
├── final_pytorch_model
├── spam_metric.log
├── requirements.txt
├── .workspace
├── plan
│   ├── plan.yaml
│   ├── cols.yaml
│   ├── defaults
│   └── data.yaml
├── logs
│   └── cnn_mnist
│       └── events.out.tfevents.1651982305.MacBook-Pro-2.local.38935.0
├── cert
├── agg_to_col_one_signed_cert.zip
├── save
│   ├── torch_cnn_mnist_best.pbuf
│   ├── torch_cnn_mnist_init.pbuf
│   └── torch_cnn_mnist_last.pbuf
├── data
│   └── MNIST
│       └── raw
├── agg_to_col_two_signed_cert.zip
└── src
    ├── mnist_utils.py
    ├── __init__.py
    ├── ptmnist_inmemory.py
    └── pt_cnn.py

9 directories, 18 files
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request and Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing Certificate CSR
3   Create Certificate Chain

Done.
Creating AGGREGATOR certificat

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [8]:
# from torch.utils.data import random_split


# def one_hot(labels, classes):
#     return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
# trainsets = random_split(trainset, [6250]*8, generator=torch.Generator().manual_seed(42))
train_data, train_labels = train_set.train_data, np.array(train_set.train_labels)
# train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_data, test_labels = test_set.test_data, np.array(test_set.test_labels)

# valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
# valid_labels = one_hot(valid_labels,10)

HTTPError: HTTP Error 503: Service Unavailable

In [4]:
# feature_shape = train_images.shape[1]
num_classes = 10

fl_data = FederatedDataSet(train_data, train_labels, test_data, test_labels, batch_size=32, num_classes=num_classes)

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
# optimizer = lambda x: optim.Adam(x, lr=1e-4)
optimizer = lambda x: optim.SGD(x, lr=0.001, momentum=0.9)
# optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    # criterion = nn.CrossEntropyLoss()
    # loss = criterion(output, target)
    # return loss
    return F.cross_entropy(input=output,target=target)

Here we can define metric logging function. It should has the following signature described below. You can use it to write metrics to tensorboard or some another specific logging.

In [5]:
# from torch.utils.tensorboard import SummaryWriter
import time

# writer = SummaryWriter('./logs/cnn_mnist', flush_secs=5)
log = open("./cifar_openfl.log", "w")

def write_metric(node_name, task_name, metric_name, metric, round_number):
    log.write("{}/{}/{}: {}, {}, {}\n".format(node_name, task_name, metric_name, metric, round_number, time.time()))
    # writer.add_scalar("{}/{}/{}".format(node_name, task_name, metric_name),
    #                   metric, round_number)

In [6]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [7]:
collaborator_models = fl_model.setup(num_collaborators=8)
collaborators = {'1':collaborator_models[0],
                 '2':collaborator_models[1],
                 '3':collaborator_models[2],
                 '4':collaborator_models[3],
                 '5':collaborator_models[4],
                 '6':collaborator_models[5],
                 '7':collaborator_models[6],
                 '8':collaborator_models[7]}

In [8]:
#Original MNIST dataset
print(f'Original training data size: {len(train_data)}')
print(f'Original validation data size: {len(test_data)}\n')

for i, model in enumerate(collaborator_models):
    print(f'Collaborator {i}\'s training data size: {len(model.data_loader.X_train)}')
    print(f'Collaborator {i}\'s validation data size: {len(model.data_loader.X_valid)}\n')

#Collaborator one's data
# print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
# print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
# print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
# print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 30000
Collaborator one's validation data size: 5000

Collaborator two's training data size: 30000
Collaborator two's validation data size: 5000



We can see the current plan values by running the `fx.get_plan()` function

In [9]:
 #Get the current values of the plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 2,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.log_metric_callback": {
        "template": "src.mnist_utils.write_metric"
    },
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.settings.write_logs": false,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
  

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [10]:
# Run experiment, return trained FederatedModel

final_fl_model = fx.run_experiment(collaborators, override_config={
    'aggregator.settings.rounds_to_train': 100,
    'aggregator.settings.log_metric_callback': write_metric,
})

/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:282: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(

/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:456: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:456: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


...

/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:456: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:456: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/Users/zhiruzhu/Desktop/data_station/fl_test/venv/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


In [11]:
#Save final model
final_fl_model.save_native('final_cifar_model')

In [ ]:
log.close()